In [40]:
import cv2
import numpy as np
from loaddata import loaddata
from sklearn import svm
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras import losses, Sequential, layers

In [34]:
x_train, x_test, y_train, y_test = loaddata('1', '2', '3', train_size=0.8)

---
### HOG

In [35]:
winSize = (160, 160) 
blockSize = (80, 80)
blockStride = (40, 40)
cellSize = (80, 80)
Bin = 9 
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, Bin)
HOG = lambda img: hog.compute(img).flatten()

train_HOG = np.array([HOG(img) for img in x_train])

SVM = svm.SVC(kernel = 'linear')
SVM.fit(train_HOG, y_train)
del train_HOG

In [36]:
test_HOG = np.array([HOG(img) for img in x_test])
y_predict = SVM.predict(test_HOG)

In [37]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92       140
           1       0.97      0.95      0.96        59
           2       0.91      0.86      0.89        37
           3       0.97      0.94      0.96        34
           4       0.95      0.85      0.90        41

    accuracy                           0.92       311
   macro avg       0.94      0.91      0.92       311
weighted avg       0.92      0.92      0.92       311



---
### CNN

In [44]:
def convModel():
    model = Sequential([
        layers.Conv2D(64, (3, 3), activation = 'relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.Flatten(),
        layers.Dense(10000, activation = 'relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation = 'softmax', name = "final")
    ])
    model.compile(optimizer = 'adam', loss = losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])
    return model

model = convModel()